# Identificar el usuario con mas commits en Git Hub

El objetivo de este cuaderno es idenfiticar el usuario con más commits durante enero de 2015. Para ello, primero iniciamos los comandos para trabajar con dask

In [1]:
from dask.distributed import Client

client = Client(n_workers=4)

Luego, hacemos la lectura de los archivos tipo json en los cuales esta toda la información. Cada archivo contiene la información con los commits realizados durante el lapso de una hora. Es decir, para un día se tienen 24 archivos y para todo el mes 744 archivos

In [2]:
import dask.bag as db
import os
#files = [os.path.join('Datos','2015-01-0%d-*.json.gz' % i) for i in range(1,2)]
files = os.path.join('Datos','*.json.gz')
b = db.read_text(files)
print(files)
print(b)

Datos/*.json.gz
dask.bag<bag-from-delayed, npartitions=744>


Ahora tomamos la primera entrada del primer archivo y la imprimimos para ver su estructura 

In [3]:
b.take(1)

('{"id":"2489368070","type":"PushEvent","actor":{"id":9152315,"login":"davidjhulse","gravatar_id":"","url":"https://api.github.com/users/davidjhulse","avatar_url":"https://avatars.githubusercontent.com/u/9152315?"},"repo":{"id":28635890,"name":"davidjhulse/davesbingrewardsbot","url":"https://api.github.com/repos/davidjhulse/davesbingrewardsbot"},"payload":{"push_id":536740396,"size":1,"distinct_size":1,"ref":"refs/heads/master","head":"a9b22a6d80c1e0bb49c1cf75a3c075b642c28f81","before":"86ffa724b4d70fce46e760f8cc080f5ec3d7d85f","commits":[{"sha":"a9b22a6d80c1e0bb49c1cf75a3c075b642c28f81","author":{"email":"da8d7d1118ca5befd4d0d3e4f449c76ba6f1ee7e@live.com","name":"davidjhulse"},"message":"Altered BingBot.jar\\n\\nFixed issue with multiple account support","distinct":true,"url":"https://api.github.com/repos/davidjhulse/davesbingrewardsbot/commits/a9b22a6d80c1e0bb49c1cf75a3c075b642c28f81"}]},"public":true,"created_at":"2015-01-01T00:00:00Z"}\n',)

Ahora, imprimimos esta registro de una mejor manera usando la librería json

In [4]:
import json
js = b.map(json.loads)
js.take(1)

({'id': '2489368070',
  'type': 'PushEvent',
  'actor': {'id': 9152315,
   'login': 'davidjhulse',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/davidjhulse',
   'avatar_url': 'https://avatars.githubusercontent.com/u/9152315?'},
  'repo': {'id': 28635890,
   'name': 'davidjhulse/davesbingrewardsbot',
   'url': 'https://api.github.com/repos/davidjhulse/davesbingrewardsbot'},
  'payload': {'push_id': 536740396,
   'size': 1,
   'distinct_size': 1,
   'ref': 'refs/heads/master',
   'head': 'a9b22a6d80c1e0bb49c1cf75a3c075b642c28f81',
   'before': '86ffa724b4d70fce46e760f8cc080f5ec3d7d85f',
   'commits': [{'sha': 'a9b22a6d80c1e0bb49c1cf75a3c075b642c28f81',
     'author': {'email': 'da8d7d1118ca5befd4d0d3e4f449c76ba6f1ee7e@live.com',
      'name': 'davidjhulse'},
     'message': 'Altered BingBot.jar\n\nFixed issue with multiple account support',
     'distinct': True,
     'url': 'https://api.github.com/repos/davidjhulse/davesbingrewardsbot/commits/a9b22a6d80c1e0bb49c1cf75a3c

Aquí podemos observar toda la información relacionada con un commit realizado el 1 de enero de 2015. Ahora, guardamos la información dataframes y miramos la cabecera del primero de ellos

In [5]:
df1 = js.to_dataframe()
df1.head()

,id,type,actor,repo,payload,public,created_at
0,2489368070,PushEvent,"{'id': 9152315, 'login': 'davidjhulse', 'grava...","{'id': 28635890, 'name': 'davidjhulse/davesbin...","{'push_id': 536740396, 'size': 1, 'distinct_si...",True,2015-01-01T00:00:00Z
1,2489368072,PushEvent,"{'id': 5581438, 'login': 'jmoon018', 'gravatar...","{'id': 26392647, 'name': 'jmoon018/rshell-unit...","{'push_id': 536740397, 'size': 1, 'distinct_si...",True,2015-01-01T00:00:00Z
2,2489368089,CreateEvent,"{'id': 6352424, 'login': 'christoferpeterson',...","{'id': 28677542, 'name': 'christoferpeterson/V...","{'ref': 'master', 'ref_type': 'branch', 'maste...",True,2015-01-01T00:00:01Z
3,2489368095,PushEvent,"{'id': 66577, 'login': 'JakeWharton', 'gravata...","{'id': 5152285, 'name': 'square/okhttp', 'url'...","{'push_id': 536740405, 'size': 8, 'distinct_si...",True,2015-01-01T00:00:01Z
4,2489368104,PushEvent,"{'id': 9221683, 'login': 'git4ruby', 'gravatar...","{'id': 28520835, 'name': 'git4ruby/movie_revie...","{'push_id': 536740413, 'size': 1, 'distinct_si...",True,2015-01-01T00:00:03Z


En este caso los commits fueron realizados el 1 de enero de 2015. Ahora si miramos la parte final de los dataframes

In [6]:
df1 = js.to_dataframe()
df1.tail()


,id,type,actor,repo,payload,public,created_at
11651,2549408785,DeleteEvent,"{'id': 7644715, 'login': 'ReChavers', 'gravata...","{'id': 27151277, 'name': 'coding-the-humanitie...","{'ref': 'style', 'ref_type': 'branch', 'pusher...",True,2015-01-31T09:59:58Z
11652,2549408786,IssueCommentEvent,"{'id': 1847482, 'login': 'HallofFamer', 'grava...","{'id': 1903522, 'name': 'php/php-src', 'url': ...","{'action': 'created', 'issue': {'url': 'https:...",True,2015-01-31T09:59:59Z
11653,2549408787,WatchEvent,"{'id': 2791964, 'login': 'fangj', 'gravatar_id...","{'id': 17762755, 'name': 'Semantic-Org/Semanti...",{'action': 'started'},True,2015-01-31T09:59:59Z
11654,2549408789,CreateEvent,"{'id': 797382, 'login': 'gitahinganga', 'grava...","{'id': 30108408, 'name': 'gitahinganga/openmrs...","{'ref': 'RA-569', 'ref_type': 'branch', 'maste...",True,2015-01-31T09:59:59Z
11655,2549408790,PushEvent,"{'id': 3426586, 'login': 'pekkalanger', 'grava...","{'id': 29687735, 'name': 'pekkalanger/PekkersL...","{'push_id': 561497280, 'size': 1, 'distinct_si...",True,2015-01-31T09:59:59Z


Aquí observamos que los commits fueron realizados el 31 de enero de 2015. De esta manera, tenemos en 744 dataframes. Para examinar esto y otra información sobre la estructura de estos dataframes usamos el siguiente comando 

In [7]:
df1.values

dask.array<values, shape=(nan, 7), dtype=object, chunksize=(nan, 7), chunktype=numpy.ndarray>

Además, miramos el tipo de datos almacenados 

In [8]:
df1.dtypes

id            object
type          object
actor         object
repo          object
payload       object
public          bool
created_at    object
dtype: object

Además, podemos obtener información sobre la forma como está indexado.

In [9]:
df1.index

Dask Index Structure:
npartitions=744
    int64
      ...
    ...  
      ...
      ...
dtype: int64
Dask Name: to_dataframe, 2976 tasks

Por otro lado, podemos ver el número de particiones y de divisiones. Las particiones corresponden al número de archivos tipos json que tenemos, y estos a su vez son iguales al número de dataframes que tenemos. Las divisiones nos dicen si dentro de cada archivo los datos están divididos y como, en este caso no tenemos divisiones

In [10]:
print(df1.npartitions)

744


In [11]:
print(df1.divisions)

(None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

Ahora, filtremos los commits que crean algún tipo de archivo o proyecto y miremos los primeros cinco

In [12]:
js.filter(lambda record: record['type'] == 'CreateEvent').take(5)

({'id': '2489368089',
  'type': 'CreateEvent',
  'actor': {'id': 6352424,
   'login': 'christoferpeterson',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/christoferpeterson',
   'avatar_url': 'https://avatars.githubusercontent.com/u/6352424?'},
  'repo': {'id': 28677542,
   'name': 'christoferpeterson/Vadek',
   'url': 'https://api.github.com/repos/christoferpeterson/Vadek'},
  'payload': {'ref': 'master',
   'ref_type': 'branch',
   'master_branch': 'master',
   'description': 'Making an HTML5 game from scratch.',
   'pusher_type': 'user'},
  'public': True,
  'created_at': '2015-01-01T00:00:01Z'},
 {'id': '2489368129',
  'type': 'CreateEvent',
  'actor': {'id': 6329667,
   'login': 'greyia',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/greyia',
   'avatar_url': 'https://avatars.githubusercontent.com/u/6329667?'},
  'repo': {'id': 27156833,
   'name': 'greyia/port2container',
   'url': 'https://api.github.com/repos/greyia/port2container'},
  'payload':

Ahora, contamos el total de commits en donde se crean archivos o proyectos

In [13]:
%%time
(js.filter(lambda record: record['type'] == 'CreateEvent').count().compute())

CPU times: user 38 s, sys: 3.87 s, total: 41.8 s
Wall time: 8min 2s


1651300

Todo lo anterior fue para hacer una breve exploración de los datos. Ahora, vamos a buscar la información relacionada con los autores de los commits que esta en la columna actor en forma de diccionario

In [14]:
def actor(d):
    return {'actor': d['actor']}

(js.filter(lambda record: record)
   .map(actor)
   .take(2))

({'actor': {'id': 9152315,
   'login': 'davidjhulse',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/davidjhulse',
   'avatar_url': 'https://avatars.githubusercontent.com/u/9152315?'}},
 {'actor': {'id': 5581438,
   'login': 'jmoon018',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/jmoon018',
   'avatar_url': 'https://avatars.githubusercontent.com/u/5581438?'}})

Esto se puede hacer de una mejor manera usando la característica pluck, pues permite extraer la información de actor como un diccionario

In [15]:
(js.filter(lambda record: record)
   .pluck('actor')
   .take(2))

({'id': 9152315,
  'login': 'davidjhulse',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/davidjhulse',
  'avatar_url': 'https://avatars.githubusercontent.com/u/9152315?'},
 {'id': 5581438,
  'login': 'jmoon018',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/jmoon018',
  'avatar_url': 'https://avatars.githubusercontent.com/u/5581438?'})

Con ello podemos encontrar el id y el login de los usuarios que hicieron commits. Sin embargo, vamos a implementar otra manera usando el comando flatten. Primero definimos una función para extraer las características que queremos y luego guardamos la información de los autores en una bolsa

In [16]:
def denormalize(record):
    return [{'id': record['actor']['id'],
            'login': record['actor']['login']}
            for id in record['id']]

autores = js.map(denormalize).flatten()
autores.take(2)

({'id': 9152315, 'login': 'davidjhulse'},
 {'id': 9152315, 'login': 'davidjhulse'})

Luego, guardamos esto en un dataframe

In [17]:
df2 = autores.to_dataframe()
df2.head()

,id,login
0,9152315,davidjhulse
1,9152315,davidjhulse
2,9152315,davidjhulse
3,9152315,davidjhulse
4,9152315,davidjhulse


Examinamos su estructura

In [18]:
df2.values

dask.array<values, shape=(nan, 2), dtype=object, chunksize=(nan, 2), chunktype=numpy.ndarray>

Ahora, contamos cuantos commits hizo cada usuario

In [19]:
%%time
df2.groupby('login').count().compute()

CPU times: user 50.5 s, sys: 5.36 s, total: 55.9 s
Wall time: 10min 52s


,id
login,
12finger,50
130s,3000
1j01,1060
309624472,40
3ft9,70
...,...
zahraebrahimi,50
zed-0xff,10
ziggyke,10


Ahora, podemos buscar la información de cada usuario para verificar. Por ejemplo el primer usuario 

In [20]:
a = autores.filter(lambda record: record['login'] == '12finger').take(1)
b = autores.filter(lambda record: record['login'] == '12finger').count().compute()
print(a,b)

({'id': 1730022, 'login': '12finger'},) 50


Con esto, guardamos esta información en un dataframe y cambiamos el nombre de la columna id a Frecuencia

In [21]:
%%time
commits = df2.groupby('login').count().compute()
commits = commits.rename(columns={'id': 'Frecuencia'})
commits

CPU times: user 1min 2s, sys: 6.59 s, total: 1min 9s
Wall time: 12min 42s


,Frecuencia
login,
12finger,50
130s,3000
1j01,1060
309624472,40
3ft9,70
...,...
zahraebrahimi,50
zed-0xff,10
ziggyke,10


Finalmente, podemos encontrar el usuario con más commits durante de enero de 2015 en Git-Hub con el siguiente comando

In [22]:
commits['Frecuencia'].idxmax()


'KenanSulayman'

In [23]:
client.shutdown()

distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError
